In [5]:
from gurobipy import Model, GRB, quicksum
import numpy as np

In [6]:
# --- Parameters ---
K = 101                          # Number of grid points
g_min, g_max = 0.0, 1.0         # SOC bounds (0–1)
Δg = (g_max - g_min) / (K - 1)  # Grid resolution
g_vals = [g_min + k * Δg for k in range(K)]  # Grid points

C = 3                           # Number of charging stops
η = 0.01                       # Efficiency loss per km
d = [100, 80]                   # Distance between c-1 and c (length C-1)

# --- Create model ---
m = Model("EV_SOC_Discretization")
m.setParam("OutputFlag", 0)  # Suppress solver output

# --- Variables ---
# SOC indicators: 1 if SOC is g_k, else 0
s_a = {(c, k): m.addVar(vtype=GRB.BINARY, name=f"s_a[{c},{k}]")
       for c in range(C) for k in range(K)}
s_l = {(c, k): m.addVar(vtype=GRB.BINARY, name=f"s_l[{c},{k}]")
       for c in range(C) for k in range(K)}

# --- Constraints ---
# Each SOC is exactly one grid point
for c in range(C):
    m.addConstr(quicksum(s_a[c, k] for k in range(K)) == 1, name=f"one_soc_a[{c}]")
    m.addConstr(quicksum(s_l[c, k] for k in range(K)) == 1, name=f"one_soc_l[{c}]")

# Arrival SOC at c = departure SOC at c-1 minus η*d
for c in range(1, C):
    lhs = quicksum(g_vals[k] * s_l[c-1, k] for k in range(K))
    rhs = quicksum(g_vals[k] * s_a[c, k] for k in range(K))

    # Arrival SOC can be snapped to either the floor or floor + Δg
    m.addConstr(lhs - η * d[c-1] - rhs >= 0, name=f"arrive_lower[{c}]")
    m.addConstr(lhs - η * d[c-1] - rhs <= Δg, name=f"arrive_upper[{c}]")

# Optional: Set initial SOC = 0
m.addConstr(quicksum(g_vals[k] * s_a[0, k] for k in range(K)) == 0.0, name="init_soc")

# --- Objective (dummy for feasibility) ---
m.setObjective(0, GRB.MINIMIZE)

# --- Solve ---
m.optimize()

# --- Results ---
print("\nOptimal SOC values:")
for c in range(C):
    soc_a = sum(g_vals[k] * s_a[c, k].x for k in range(K))
    soc_l = sum(g_vals[k] * s_l[c, k].x for k in range(K))
    print(f"Stop {c}: SOC^a = {soc_a:.3f}, SOC^l = {soc_l:.3f}")



Optimal SOC values:
Stop 0: SOC^a = 0.000, SOC^l = 1.000
Stop 1: SOC^a = 0.000, SOC^l = 0.810
Stop 2: SOC^a = 0.000, SOC^l = 0.000


In [3]:
from gurobipy import Model, GRB, quicksum

# --- Parameters ---
K = 101                          # Number of grid points
g_min, g_max = 0.0, 1.0         # SOC bounds (0–1)
delta_g = (g_max - g_min) / (K - 1)  # Grid resolution
g_vals = [g_min + k * delta_g for k in range(K)]  # Grid points

C = 3                           # Number of charging stops
T = 10                          # Number of time slots per stop
eta = 0.01                      # Efficiency loss per km
d = [100, 80]                   # Distance between c-1 and c (length C-1)

# --- Create model ---
m = Model("EV_SOC_Discretization_with_X")
m.setParam("OutputFlag", 0)  # Suppress solver output

# --- Variables ---
# SOC indicators
s_a = {(c, k): m.addVar(vtype=GRB.BINARY, name=f"s_a[{c},{k}]")
       for c in range(C) for k in range(K)}
s_l = {(c, k): m.addVar(vtype=GRB.BINARY, name=f"s_l[{c},{k}]")
       for c in range(C) for k in range(K)}

# Charging decision variables
x = {(c, t): m.addVar(vtype=GRB.BINARY, name=f"x[{c},{t}]")
     for c in range(C) for t in range(T)}

# --- Constraints ---
# Each SOC is exactly one grid point
for c in range(C):
    m.addConstr(quicksum(s_a[c, k] for k in range(K)) == 1, name=f"one_soc_a[{c}]")
    m.addConstr(quicksum(s_l[c, k] for k in range(K)) == 1, name=f"one_soc_l[{c}]")

# Arrival SOC at c = departure SOC at c-1 minus eta*d
for c in range(1, C):
    lhs = quicksum(g_vals[k] * s_l[c-1, k] for k in range(K))
    rhs = quicksum(g_vals[k] * s_a[c, k] for k in range(K))
    m.addConstr(lhs - eta * d[c-1] - rhs >= 0, name=f"arrive_lower[{c}]")
    m.addConstr(lhs - eta * d[c-1] - rhs <= delta_g, name=f"arrive_upper[{c}]")

# Initial SOC = 0
m.addConstr(quicksum(g_vals[k] * s_a[0, k] for k in range(K)) == 0.0, name="init_soc")

# --- Dummy Objective: Minimize total time charging ---
m.setObjective(quicksum(x[c, t] for c in range(C) for t in range(T)), GRB.MINIMIZE)

# --- Solve ---
m.optimize()

# --- Results ---
print("\nOptimal SOC values:")
for c in range(C):
    soc_a = sum(g_vals[k] * s_a[c, k].x for k in range(K))
    soc_l = sum(g_vals[k] * s_l[c, k].x for k in range(K))
    print(f"Stop {c}: SOC^a = {soc_a:.3f}, SOC^l = {soc_l:.3f}")

print("\nCharging decisions:")
for c in range(C):
    for t in range(T):
        if x[c, t].x > 0.5:
            print(f"Charge at stop {c} during time {t}")



Optimal SOC values:
Stop 0: SOC^a = 0.000, SOC^l = 1.000
Stop 1: SOC^a = 0.000, SOC^l = 0.810
Stop 2: SOC^a = 0.000, SOC^l = 0.000

Charging decisions:


In [4]:
import gurobipy as gp
from gurobipy import GRB

# Parameters
C = 3                          # Number of charging stops
K = 101                        # Number of SOC grid points (0.00 to 1.00 by 0.01)
T = 20                         # Total number of time steps available
eta = 0.25                     # SOC consumption per unit distance
delta_g = 0.05                 # SOC gained per charging time step

distances = [1.0, 2.0, 1.0]    # Distances between stops
g_vals = [k / (K - 1) for k in range(K)]  # SOC grid: 0.00, 0.01, ..., 1.00

# Gurobi model
m = gp.Model("EV_Charging")

# Variables
x = m.addVars(C, T, vtype=GRB.BINARY, name="x")  # x[c,t] = 1 if charge at stop c during time t
s_a = m.addVars(C + 1, K, vtype=GRB.BINARY, name="s_a")  # SOC^a[c] at arrival
s_l = m.addVars(C, K, vtype=GRB.BINARY, name="s_l")      # SOC^l[c] after charging

# Each SOC must be exactly one grid value
for c in range(C + 1):
    m.addConstr(gp.quicksum(s_a[c, k] for k in range(K)) == 1, name=f"soc_a_unique_{c}")
for c in range(C):
    m.addConstr(gp.quicksum(s_l[c, k] for k in range(K)) == 1, name=f"soc_l_unique_{c}")

# Initial SOC
m.addConstr(s_a[0, int(0.5 * (K - 1))] == 1, name="initial_SOC")  # SOC^a[0] = 0.5

# SOC transition constraints
for c in range(C):
    # Link charging decisions to SOC^l
    m.addConstr(
        gp.quicksum(x[c, t] for t in range(T)) * delta_g
        + gp.quicksum(g_vals[k] * s_a[c, k] for k in range(K))
        ==
        gp.quicksum(g_vals[k] * s_l[c, k] for k in range(K)),
        name=f"soc_l_link_{c}"
    )

    # SOC consumption due to travel (remove +delta_g to avoid infeasibility)
    m.addConstr(
        gp.quicksum(g_vals[k] * s_a[c + 1, k] for k in range(K))
        >=
        gp.quicksum(g_vals[k] * s_l[c, k] for k in range(K)) - eta * distances[c],
        name=f"soc_a_transition_{c}"
    )

# Objective: minimize total time spent charging
m.setObjective(gp.quicksum(x[c, t] for c in range(C) for t in range(T)), GRB.MINIMIZE)

# Optimize
m.optimize()

# If infeasible, diagnose
if m.status == GRB.INFEASIBLE:
    print("\nModel is infeasible. Writing .ilp file for inspection...")
    m.computeIIS()
    m.write("model.ilp")
else:
    # Output charging schedule
    print("\nCharging schedule (x[c, t] == 1):")
    for c in range(C):
        charging_times = [t for t in range(T) if x[c, t].X > 0.5]
        print(f"Stop {c}: {charging_times} (Total: {len(charging_times)})")

    # Output SOC values
    print("\nOptimal SOC values:")
    for c in range(C):
        soc_a_val = sum(g_vals[k] * s_a[c, k].X for k in range(K))
        soc_l_val = sum(g_vals[k] * s_l[c, k].X for k in range(K))
        print(f"Stop {c}: SOC^a = {soc_a_val:.2f}, SOC^l = {soc_l_val:.2f}")
    soc_final = sum(g_vals[k] * s_a[C, k].X for k in range(K))
    print(f"Stop {C}: SOC^a = {soc_final:.2f}")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21G115)

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 14 rows, 767 columns and 1968 nonzeros
Model fingerprint: 0x0ba8e508
Variable types: 0 continuous, 767 integer (767 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+00]
Presolve removed 4 rows and 405 columns
Presolve time: 0.03s
Presolved: 10 rows, 362 columns, 947 nonzeros
Variable types: 0 continuous, 362 integer (359 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.02 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
